In [44]:
import os
import re
from langchain.text_splitter import TextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import FAISS

import pandas as pd
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
from langchain.schema import Document
import weaviate

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [20]:
WEAVIATE_API_KEY = os.getenv('WEAVIATE_API_KEY2')

In [21]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## Load Docs

In [12]:
loader = DirectoryLoader('top_50')

In [13]:
%time pages = loader.load()

CPU times: user 1h 31min 46s, sys: 4min, total: 1h 35min 47s
Wall time: 2h 1min 33s


In [14]:
len(pages)

42

## Custom Splitting

In [22]:
import re
import pandas as pd
from bs4 import BeautifulSoup

class CustomTextSplitter(TextSplitter):

    def split_text(self, document):
        text = document.page_content

        metadata = {
            "company": self.extract_metadata(r'COMPANY CONFORMED NAME:\s*([^\n]+)', text),
            "date": self.extract_metadata(r'FILED AS OF DATE:\s*(\d+)', text),
            "sic": self.extract_metadata(r'STANDARD INDUSTRIAL CLASSIFICATION:\s*([^\n]+)', text),
            "state": self.extract_metadata(r'STATE:\s*([^\n]+)', text)
        }

        ten_k_text = self.extract_10k(text)

        sections_df = self.get_sections_dataframe(ten_k_text)
        # display(sections_df)

        return self.extract_cleaned_sections(ten_k_text, sections_df, metadata)

    def extract_metadata(self, pattern, text):
        match = re.search(pattern, text)
        return match.group(1).strip() if match else None

    def extract_10k(self, text):
        doc_start_pattern = re.compile(r'<DOCUMENT>')
        doc_end_pattern = re.compile(r'</DOCUMENT>')
        type_pattern = re.compile(r'<TYPE>[^\n]+')

        doc_start_is = [x.end() for x in doc_start_pattern.finditer(text)]
        doc_end_is = [x.start() for x in doc_end_pattern.finditer(text)]
        doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(text)]

        for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
            if doc_type == '10-K':
                return text[doc_start:doc_end]

        return None

    def get_sections_dataframe(self, ten_k_text):
        regex = re.compile(r'(>Item(\s|&#160;|&nbsp;)(1.|1A|1B|2|3|4|5|6|7.|7A|8.|9A|9B|9.|[1][0-5])\.{0,1})|(ITEM(\s|&#160;|&nbsp;)(1.|1A|1B|2|3|4|5|6|7.|7A|8.|9A|9B|9.|[1][0-5])\.{0,1})')
        matches = regex.finditer(ten_k_text)

        df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches], columns=['item', 'start', 'end'])
        df['item'] = df['item'].str.lower().str.replace('&#160;|&nbsp;| |\.', '', regex=True).str.replace('>', '', regex=False)

        # Filter for the last occurrence of each section
        df = df.groupby('item').last().reset_index()

        # Map sections to a predefined order
        section_order = {
            'item1': 1, 'item1a': 2, 'item1b': 3, 'item2': 4, 'item3': 5, 'item4': 6,
            'item5': 7, 'item6': 8, 'item7': 9, 'item7a': 10, 'item8': 11, 'item9': 12,
            'item9a': 13, 'item9b': 14, 'item9c': 15, 'item10': 16, 'item11': 17, 'item12': 18,
            'item13': 19, 'item14': 20, 'item15': 21, 'item16': 22
        }
        df['order'] = df['item'].map(section_order)

        # Sort by predefined order
        return df.sort_values(by=['order']).reset_index(drop=True)


    def extract_cleaned_sections(self, ten_k_text, sections_df, metadata):
        sections = ['item1', 'item1a', 'item1b', 'item2', 'item3', 'item4', 'item5', 'item6', 'item7',
                    'item7a', 'item8', 'item9', 'item9a', 'item9b', 'item9c', 'item10', 'item11', 'item12',
                    'item13', 'item14', 'item15', 'item16']
        section_titles = {
                    'item1': 'Business',
                    'item1a': 'Risk Factors',
                    'item1b': 'Unresolved Staff Comments',
                    'item2': 'Properties',
                    'item3': 'Legal Proceedings',
                    'item4': 'Mine Safety Disclosures',
                    'item5': 'Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities',
                    'item6': 'Selected Financial Data',
                    'item7': 'Management’s Discussion and Analysis of Financial Condition and Results of Operations',
                    'item7a': 'Quantitative and Qualitative Disclosures About Market Risk',
                    'item8': 'Financial Statements and Supplementary Data',
                    'item9': 'Changes in and Disagreements With Accountants on Accounting and Financial Disclosure',
                    'item9a': 'Controls and Procedures',
                    'item9b': 'Other Information',
                    'item10': 'Directors, Executive Officers and Corporate Governance',
                    'item11': 'Executive Compensation',
                    'item12': 'Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters',
                    'item13': 'Certain Relationships and Related Transactions, and Director Independence',
                    'item14': 'Principal Accountant Fees and Services',
                    'item15': 'Exhibits and Financial Statement Schedules',
                    'item16': 'Form 10-K Summary'
                }

        cleaned_sections = []

        for index, section_name in enumerate(sections[:-1]):  # We use -1 because we don't need to process the last item separately
            current_start_values = sections_df[sections_df['item'] == section_name]['start'].values
            if len(current_start_values) == 0:  # section_name not found in the dataframe
                continue
            current_start = current_start_values[0]

            # Adjusting for the next section's start
            if index + 1 < len(sections):
                next_start_values = sections_df[sections_df['item'] == sections[index+1]]['start'].values
                if len(next_start_values) == 0:  # next section name not found in the dataframe
                    next_start = len(ten_k_text)
                else:
                    next_start = next_start_values[0]
            else:
                next_start = len(ten_k_text)

            section_title = section_titles.get(section_name, "")
            section_content = ten_k_text[current_start:next_start]

            current_metadata = metadata.copy()
            current_metadata["title"] = section_title

            content = BeautifulSoup(section_content, "lxml").get_text("\n\n")

            cleaned_sections.append({
                "content": content,
                "metadata": current_metadata
            })

        return cleaned_sections

In [23]:
custom_splitter = CustomTextSplitter()

In [24]:
# Split on sections of the 10-k
sections = []
for page in pages:
    transformed_doc = custom_splitter.split_text(page)
    sections.extend(transformed_doc)

In [26]:
# Break up sections into chunks
chunk_size = 1000
chunk_overlap = 200
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# Create an empty list to store the documents
documents = []

# Iterate over the sections and split large sections into smaller chunks
for section in sections:
    if len(section['content']) > chunk_size:
        chunks = text_splitter.split_text(section['content'])
        # Append the section's metadata to each chunk
        for chunk in chunks:
            modified_chunk = {
                "content": chunk,
                "metadata": section["metadata"]
            }
            # Create a Document object for each chunk and add it to the documents list
            document = Document(page_content=modified_chunk['content'], metadata=modified_chunk['metadata'])
            documents.append(document)
    else:
        # Create a Document object for the section and add it to the documents list
        document = Document(page_content=section['content'], metadata=section['metadata'])
        documents.append(document)

In [27]:
len(documents)

29185

In [32]:
documents[0]

Document(page_content='>Item\xa01.\xa0\xa0\xa0\xa0Business\n\nGeneral\n\nVerizon\n\nCommunications Inc. (the Company) is a holding company that, acting through its subsidiaries (together with the Company, collectively, Verizon), is one of the world’s leading providers of communications, technology, information and entertainment products and services to consumers, businesses and government entities.\n\nWith a presence around the world, we offer data, video and voice services and solutions on our networks and platforms that are designed to meet customers’ demand for mobility, reliable network connectivity, security and control.\n\nOur principal executive offices are located at 1095 Avenue of the Americas, New York, New York 10036 (telephone number 212-395-1000).\n\nWe have two reportable segments that we operate and manage as strategic business units - Verizon Consumer Group (Consumer) and Verizon Business Group (Business).\n\nVerizon Consumer Group', metadata={'company': 'VERIZON COMMUN

## Vector Store

In [29]:
auth_config = weaviate.AuthApiKey(api_key=WEAVIATE_API_KEY)  # Replace w/ your Weaviate instance API key

# Instantiate the client
client = weaviate.Client(
    url="https://sec-weaviate-4syafahy.weaviate.network", # Replace w/ your Weaviate cluster URL
    auth_client_secret=auth_config,
    additional_headers={
        "X-OpenAI-Api-Key": "YOUR-OPENAI-API-KEY", # Replace with your OpenAI key
        }
)

In [33]:
client.is_ready()

True

In [35]:
embeddings = OpenAIEmbeddings()

In [43]:
try:
    vector_store = Weaviate.from_documents(documents, embeddings, client=client)
except ConnectionError as e:
    print(f"Unexpected status code: {e.status_code}, with response body: {e.response_body}")

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-hIGwihMGYwCXLfkEAXet778V on tokens per min. Limit: 1000000 / min. Current: 922745 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-hIGwihMGYwCXLfkEAXet778V on tokens per min. Limit: 1000000 / min. Current: 842408 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-hIGwihMGYwCXLfkEAXet778V on tokens per min. Limit: 

ConnectionError: Schema could not be retrieved.

In [47]:
try:
    vectordb = FAISS.from_documents(documents, embeddings)
except ConnectionError as e:
    print(f"Unexpected status code: {e.status_code}, with response body: {e.response_body}")

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-hIGwihMGYwCXLfkEAXet778V on tokens per min. Limit: 1000000 / min. Current: 848208 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-hIGwihMGYwCXLfkEAXet778V on tokens per min. Limit: 1000000 / min. Current: 892640 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-hIGwihMGYwCXLfkEAXet778V on tokens per min. Limit: 

In [52]:
vectordb.save_local("faiss_index")

print('Local FAISS index has been successfully saved.')

Local FAISS index has been successfully saved.


In [49]:
# Example Search
query = "What products does Apple sell?"
docs = vectordb.similarity_search(query)

In [50]:
len(docs)

4

In [51]:
docs[0]

Document(page_content='product.\n\n(2)\n\nWearables, Home and Accessories net sales include sales of AirPods, Apple TV, Apple Watch, Beats products, HomePod mini and accessories.\n\n(3)\n\nServices net sales include sales from the Company’s advertising, AppleCare, cloud, digital content, payment and other services.\n\nServices net sales also include amortization of the deferred value of services bundled in the sales price of certain products.\n\niPhone\n\niPhone net sales increased during 2022 compared to 2021 due primarily to higher net sales from the Company’s new iPhone models released since the beginning of the fourth quarter of 2021.\n\nMac\n\nMac net sales increased during 2022 compared to 2021 due primarily to higher net sales of laptops.\n\niPad\n\niPad net\n\nsales decreased during 2022 compared to 2021 due primarily to lower net sales of iPad Pro.\n\nWearables, Home and Accessories', metadata={'company': 'Apple Inc.', 'date': '20221028', 'sic': 'ELECTRONIC COMPUTERS [3571]', 